In [1]:
import os

data_dir = "./mini_parsed_data.csv"

if not os.path.exists(data_dir):
  !wget https://raw.githubusercontent.com/stran123/6.864-final-project/main/data/mini_parsed_data.csv

--2021-05-03 05:57:36--  https://raw.githubusercontent.com/stran123/6.864-final-project/main/data/mini_parsed_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22310599 (21M) [text/plain]
Saving to: ‘mini_parsed_data.csv’

mini_parsed_data.cs 100%[===================>]  21.28M  45.4MB/s    in 0.5s    

2021-05-03 05:57:37 (45.4 MB/s) - ‘mini_parsed_data.csv’ saved [22310599/22310599]



In [3]:
import pandas as pd
import spacy
from tqdm.notebook import tqdm

df = pd.read_csv(data_dir)
df = df[['conversation_id','message','parsed_msg']][:100]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
START=['[START]']
PAD=['[PAD]']

longest_msg = max(df['parsed_msg'], key=lambda x: len(x.split(' '))).split(' ')

train_data = []
valid_data = []
vocab = {START[0], PAD[0]}
prev_conv_id = 0
training_over = False

for conv_id, msg, parsed_msg in df.values:
  if conv_id > 3:
    training_over = True

  if prev_conv_id != conv_id:
    if not training_over:
      train_data.append(START+PAD*(len(longest_msg)-1))
    else:
      valid_data.append(START+PAD*(len(longest_msg)-1))
    prev_conv_id = conv_id

  parsed_msg_split = parsed_msg.split(' ')
  vocab|=set(parsed_msg_split)

  if not training_over:
    train_data.append(parsed_msg.split(' ')+PAD*(len(longest_msg)-len(parsed_msg_split)))
  else:
    valid_data.append(parsed_msg.split(' ')+PAD*(len(longest_msg)-len(parsed_msg_split)))

In [38]:
train_data_encoded = []
vocab_oh = {w:i for i,w in enumerate(vocab)}
for msg in train_data:
  msg_vectorized = []
  for word in msg:
    msg_vectorized.append(vocab_oh[word])
  train_data_encoded.append(msg_vectorized)

In [14]:
# tokenizer = spacy.load('en_core_web_sm')

# tokenized_msgs = []

# df['parsed_msg'] = None
# for i, msg in tqdm(enumerate(df['message'].values[:100])):
#   tokenized_msg = tokenizer(msg[1:])
#   df['parsed_msg'][i]=' '.join([token.text for token in tokenized_msg])

# df.to_csv('mini_parsed_data.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# data = []
# prev_conv_id = 0
# for conv_id, msg in df.values:
#   if prev_conv_id != conv_id:
#     data.append('[START]')
#     prev_conv_id = conv_id
#   data.append(msg[1:])

In [25]:
import torch
import torch.nn as nn
class Embedder(nn.Module):
  def __init__(self, num_words, num_layers, hidden_size, dropout=0):
    super(Embedder, self).__init__()

    self.embedding = nn.Embedding(num_words, hidden_size)

    self.lstm = nn.LSTM(input_size=hidden_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        batch_first=True,
                        dropout=dropout,
                        bidirectional=True)
  
  def forward(self, x):
    x = self.embedding(x)
    outputs, hidden = self.lstm(x)
    return outputs

In [26]:
class Predictor(nn.Module):
    
  def __init__(self, num_words, num_layers, hidden_size, dropout=0):
    super(Predictor, self).__init__()
    self.embedder = Embedder(num_words, num_layers, hidden_size)
    self.dropout = nn.Dropout(dropout)
    self.lstm = nn.LSTM(input_size=hidden_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        batch_first=True,
                        dropout=dropout,
                        bidirectional=True)
  
  def forward(self, x):
    word_embeddings = self.embedder(x)[0]
    word_embeddings = self.dropout(word_embeddings)
    return self.lstm(word_embeddings)

In [61]:
model = Predictor(len(vocab), 1, 1024)
# batch_size = 
input = torch.tensor(train_data_encoded[0])

input=input.unsqueeze(1)
print(input.shape)
input=input.unsqueeze(2)
print(input.shape)
model(torch.tensor(input))
# torch.tensor(input).shape

torch.Size([37, 1])
torch.Size([37, 1, 1])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


RuntimeError: ignored